In [2]:
import pandas as pd

In [3]:
def ganancia(model, X, y, prop):
    y_pred = model.predict(X)
    return f1_score(y, y_pred)

In [4]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

In [ ]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    breakpoints = sorted(set(
    [edge.left for edge in bin_edges if isinstance(edge, pd.Interval)] + 
    [edge.right for edge in bin_edges if isinstance(edge, pd.Interval)]))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [6]:
# Creador clase ternaria

# df_parallel = pd.read_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\Git\dmeyf2024\datasets\competencia_01_crudo.csv")

# start_time = time.time()

# # Crear la columna clase_ternaria usando la columna 'foto_mes' basada en la presencia real

# df_ternaria= duckdb.query('''

#     WITH siguiente_mes AS (

#         SELECT

#             numero_de_cliente,

#             foto_mes,

#             -- Obtener los meses siguientes en los que el cliente está presente

#             LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo1,

#             LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo2

#         FROM df_parallel

#     )

#     SELECT

#         numero_de_cliente,

#         foto_mes,

#         CASE

#             -- Si el cliente está en los dos meses consecutivos siguientes

#             WHEN foto_mes_proximo1 = foto_mes + 1 AND foto_mes_proximo2 = foto_mes + 2 THEN 'CONTINUA'

#             -- Si el cliente no está en el siguiente mes

#             WHEN foto_mes_proximo1 IS NULL OR foto_mes_proximo1 <> foto_mes + 1 THEN 'BAJA+1'

#             -- Si el cliente está en el siguiente mes pero no en el segundo mes consecutivo

#             WHEN foto_mes_proximo1 = foto_mes + 1 AND (foto_mes_proximo2 IS NULL OR foto_mes_proximo2 <> foto_mes + 2) THEN 'BAJA+2'

#             ELSE NULL

#         END AS clase_ternaria

#     FROM siguiente_mes

#     ORDER BY numero_de_cliente, foto_mes

#     ''')

# df_ternaria = df_ternaria.to_df()
# df_ternaria = df_ternaria[['numero_de_cliente', 'foto_mes', 'clase_ternaria']]


# df_parallel = df_parallel.merge(df_ternaria, on=['numero_de_cliente', 'foto_mes'], how='left')

# df_train = df_parallel


In [7]:

# con = duckdb.connect(database=':memory:', read_only=False)

# ruta_csv = r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\datasets\competencia_2\competencia_02_f_e_lags.parquet"

# con.execute(f"CREATE TABLE df_train AS SELECT * FROM read_parquet('{ruta_csv}')")

In [ ]:
def calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test):

    lista_medidas = [accuracy_score, precision_score, recall_score, f1_score, roc_auc_score]
    train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)
    test_data = lgb.Dataset(X_test, label=X_test, weight=w_test)
    df_voting = pd.DataFrame()

    for semilla in semillas:
        params = {
            'objective': 'binary',  
            'metric': 'binary_logloss',  
            'seed': semilla, }
        model_default = lgb.train(params, train_data)   
        
        df_voting[f'semilla_{semilla}'] = model_default.predict(X_test)

    df_voting['promedio'] = df_voting.mean(axis=1)
    y_pred_default = df_voting['promedio']
    y_pred_labels = (y_pred_default >= 0.025).astype(int)
    dicc_medidas = {}
    
    for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
    ganancia = lgb_gan_eval(y_pred_default, test_data)
    dicc_medidas['ganancia'] = ganancia[1]
    
    return dicc_medidas

In [2]:
def anotar_excel(path_excel, dicc_medidas):
    if excel == True:
    
        wb = load_workbook(path_excel)
        ws = wb.active

        header_row = 1  
        column_mapping = {cell.value: cell.column for cell in ws[header_row]}
    
    row_to_update = None

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=1):
        cell = row[0]  # La primera celda de la fila
        if cell.value is None:  # Si el valor de la celda está vacía
            print(f"Primera celda vacía en la fila {cell.row}")
            row_to_update = cell.row
            break

    # Si no se encontró ninguna celda vacía, agregar al final
    if row_to_update is None:
        print("No se encontro celda vacía, agregando al final")
        row_to_update = ws.max_row + 1  # Agregar en una nueva fila al final

    print(f"Fila a actualizar: {row_to_update}")
    fila_actualizar = row_to_update  # Fila donde escribir los datos

    for col_name, value in dicc_medidas.items():
        if col_name in column_mapping:  # Asegurarse de que la columna existe
            col_idx = column_mapping[col_name]
            ws.cell(row=fila_actualizar, column=col_idx, value=value)

    # Guardar los cambios
    wb.save(path_excel)
    
    return print (f"se actualizó fila {fila_actualizar} en el excel")

In [ ]:
def calcular_medidas(X_train, Y_train, w_train, X_test, Y_test, w_test):

    lista_medidas = [accuracy_score, precision_score, recall_score, f1_score, roc_auc_score]
    train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)
    test_data = lgb.Dataset(X_test, label=Y_test, weight=w_test)
    df_voting = pd.DataFrame()

    for semilla in semillas:
        params = {
            'objective': 'binary',  
            'metric': 'binary_logloss',  
            'seed': semilla, }
        model_default = lgb.train(params, train_data)   
        
        df_voting[f'semilla_{semilla}'] = model_default.predict(X_test)

    df_voting['promedio'] = df_voting.mean(axis=1)
    y_pred_default = df_voting['promedio']
    y_pred_labels = (y_pred_default >= 0.025).astype(int)
    
    dicc_medidas = {}
    
    for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
     
    ganancia = lgb_gan_eval(y_pred_default, test_data)
    dicc_medidas['ganancia'] = ganancia[1]
    
    return dicc_medidas

In [1]:
def calculoGanancia(bajas,prediccion,corte,seed):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia para el público.
    ganancia_privado: Ganancia para el privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=seed
    )

    # Clientes que decido estimular
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Primero, calculamos la ganancia en cada conjunto
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Luego, normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado


In [1]:
def predecir_punto_de_corte (dataset, mes_de_entrenamiento, mes_de_prueba, best_params, best_iteration, rango_cortes, semillas):
    if isinstance(dataset, pd.DataFrame):
        df_train = pd.read_parquet(dataset_path)
    elif isinstance(dataset, str):
        df_train = pd.read_parquet(dataset)
    else: print('dataset debe ser un DataFrame o un string con el path del archivo')
    
    ganancia_acierto = 273000
    costo_estimulo = 7000
    mes_train = mes_de_entrenamiento
    mes_test = mes_de_prueba
    data = df_train
    data['clase_peso'] = 1.0
    data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
    data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
    data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
    data['foto_mes'].unique()
    if isinstance(mes_de_entrenamiento, list):
        df_train = data[data['foto_mes'].isin(mes_train)]
    else: df_train = data[data['foto_mes']==mes_train]
    df_test = data[data['foto_mes']==mes_test]
    clase_peso = df_train['clase_peso']
    X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
    Y_train =df_train['clase_binaria']
    X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
    Y_test =df_test['clase_binaria']
    w_train = df_train['clase_peso']
    w_test = df_test['clase_peso']
    best_params = best_params
    best_iter = best_iteration    
    train_data = lgb.Dataset(X_train,
                                label=Y_train,
                                weight=w_train)
    best_iter = 1789

    df_voting = pd.DataFrame()

    for x in semillas:
        params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': best_params['num_leaves'],
        'learning_rate': best_params['learning_rate'],
        'min_data_in_leaf': best_params['min_data_in_leaf'],
        'feature_fraction': best_params['feature_fraction'],
        'bagging_fraction': best_params['bagging_fraction'],
        'seed': x,
        'verbose': 0}
            
                
        model = lgb.train(params,
                        train_data,
                        num_boost_round=best_iter)

        y_pred = model.predict(X_test)
        df_voting[f'prediccion_seed_{x}'] = y_pred

    df_voting['prediccion'] = df_voting.mean(axis=1)
    df_voting.index = X_test.index
    bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']}, index= X_test.index)
    prediccion = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'probabilidad': df_voting ['prediccion']}, index=X_test.index) 
    resultados = []
    cortes = range(5000, 20000, rango_cortes)


    pred_model_sorted = prediccion.sort_values('probabilidad', ascending=False)

    model_name = 'LightGBM'

    # Iteramos sobre cada corte
    for corte in cortes:
        lista_ganancia_publico =[]
        lista_ganancia_privado = []
        for semilla in semillas: 
            ganancia_publico, ganancia_privado = calculoGanancia(bajas, pred_model_sorted, corte, semilla)        
            lista_ganancia_publico.append(ganancia_publico)
            lista_ganancia_privado.append(ganancia_privado)
        promedio_ganancia_publico = np.mean(lista_ganancia_publico)
        promedio_ganancia_privado = np.mean(lista_ganancia_privado)
            
        resultados.append({
        'Modelo': model_name,
        'Corte': corte,
        'Ganancia Público': promedio_ganancia_publico,
        'Ganancia Privado': promedio_ganancia_privado
        })

    # Convertimos los resultados en un DataFrame
    resultados = pd.DataFrame(resultados)
    resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado'])

    # Aplanamos las columnas para facilitar el acceso
    resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

    # Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
    # Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
    columnas_ordenadas = []
    for modelo in resultados['Modelo'].unique():
        columnas_ordenadas.append(f'Ganancia Público_{modelo}')
        columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

    # Reorganizamos el DataFrame usando el nuevo orden de columnas
    resultados_pivot = resultados_pivot[columnas_ordenadas]

    # Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
    resultados_pivot = resultados_pivot.reset_index()
    maxima_ganancia_publico = resultados_pivot['Ganancia Público_LightGBM'].max()
    maxima_ganancia_privado = resultados_pivot['Ganancia Privado_LightGBM'].max()


    for indice, resultado in resultados_pivot.iterrows():
        if resultado['Ganancia Público_LightGBM'] == maxima_ganancia_publico:
            corte_maxima_ganancia_publico = indice  # Puedes ajustar según la lógica de "corte"
            ganancia_si_corte_maxima_ganancia_publico = (
                resultado['Ganancia Público_LightGBM'] + resultado['Ganancia Privado_LightGBM']
            )
            print(f'Ganancia si se hace corte por máxima ganancia público: {ganancia_si_corte_maxima_ganancia_publico}, el corte será {corte_maxima_ganancia_publico}')
        
        if resultado['Ganancia Privado_LightGBM'] == maxima_ganancia_privado:
            corte_maxima_ganancia_privado = indice  # Puedes ajustar según la lógica de "corte"
            ganancia_si_corte_maxima_ganancia_privado = (
                resultado['Ganancia Público_LightGBM'] + resultado['Ganancia Privado_LightGBM']
            )
            print(f'Ganancia si se hace corte por máxima ganancia privado: {ganancia_si_corte_maxima_ganancia_privado}, el corte será {corte_maxima_ganancia_privado}')
                
    corte_publico_ganancia_total = resultados_pivot.loc[corte_maxima_ganancia_publico,'Ganancia Privado_LightGBM'] + resultados_pivot.loc[corte_maxima_ganancia_publico,'Ganancia Público_LightGBM']
    corte_privado_ganancia_total = resultados_pivot.loc[corte_maxima_ganancia_privado,'Ganancia Privado_LightGBM'] + resultados_pivot.loc[corte_maxima_ganancia_privado,'Ganancia Público_LightGBM']
    corte_maxima_ganancia_total = corte_maxima_ganancia_privado if corte_privado_ganancia_total > corte_publico_ganancia_total else corte_maxima_ganancia_publico
    incentivos = resultados_pivot.loc[corte_maxima_ganancia_total,'Corte']
    print (f'El corte recomendado por máxima ganancia total es {corte_maxima_ganancia_total} con un incentivo de {incentivos}')
    return incentivos, resultados_pivot

In [19]:
def hacer_anotacion_corte (dataset:str, extra:str, df_train:pd.DataFrame, incentivos:int, mensaje:str):
    """
    variables: fecha,hora,dataset,cantidad_variables,corte,extra
    """
    
    fecha = datetime.now().strftime("%Y-%m-%d")
    hora = datetime.now().strftime("%H:%M:%S")
    cantidad_variables = df_train.shape
    
    dataframe = pd.read_csv(r"../../anotaciones_punto_de_corte.csv")
    nueva_fila = pd.DataFrame({'fecha':[fecha],'hora':[hora],'dataset':[dataset],'cantidad_variables':[cantidad_variables],'corte':[incentivos],'extra':[extra],'mensaje':[mensaje]})
    dataframe = pd.concat([dataframe, nueva_fila], ignore_index=True)
    dataframe.to_csv(r"../../anotaciones_punto_de_corte.csv", index=False)
    os.system('git add ../../anotaciones_punto_de_corte.csv')
    os.system(f"git commit -m '{mensaje}'")
    os.system("git push")

In [18]:
def hacer_anotacion_back_testing (dataset:str, extra:str, df_train:pd.DataFrame, continuas:int, baja_1:int, baja_2:int , mensaje:str):
    """
    variables: fecha,hora,dataset,continuas,baja+1,baja+2,corte,extra
    """
    
    fecha = datetime.now().strftime("%Y-%m-%d")
    hora = datetime.now().strftime("%H:%M:%S")
    cantidad_variables = df_train.shape
    
    dataframe = pd.read_csv(r"../../anotaciones_back_testing.csv")
    nueva_fila = pd.DataFrame({'fecha':[fecha],'hora':[hora],'dataset':[dataset],'cantidad_variables':[cantidad_variables],'continuas':[continuas], 'baja+1':[baja_1],'extra':[baja_2],'mensaje':[mensaje]})
    dataframe = pd.concat([dataframe, nueva_fila], ignore_index=True)
    dataframe.to_csv(r"../../anotaciones_back_testing.csv", index=False)
    os.system('git add ../../anotaciones_back_testing.csv')
    os.system(f"git commit -m '{mensaje}'")
    os.system("git push")